In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Get mask store info from API

In [2]:
import requests 
import json 
from tqdm import tqdm 
import pandas as pd

In [3]:
def getDistrictMaskStoreInfo(address): 
  url = "https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/storesByAddr/json?address=" + requests.utils.unquote(address) 
  
  req = requests.get(url) 
  
  json_data = req.json() 
  store_data = json_data['stores'] 
  
  addrs = [] 
  codes = [] 
  latitudes = [] 
  longitudes = [] 
  names = [] 
  types = [] 
  created_ats = [] 
  remain_stats = [] 
  stock_ats = [] 
  
  for i in tqdm(range(len(store_data))): 
    addrs.append(store_data[i]['addr']) 
    codes.append(store_data[i]['code']) 
    latitudes.append(store_data[i]['lat']) 
    longitudes.append(store_data[i]['lng']) 
    names.append(store_data[i]['name']) 
    types.append(store_data[i]['type']) 
    try: 
      created_ats.append(store_data[i]['created_at']) 
    except: 
      created_ats.append("no_data") 
    try: 
      remain_stats.append(store_data[i]['remain_stat']) 
    except: 
      remain_stats.append("no_data")
    try: 
      stock_ats.append(store_data[i]['stock_at']) 
    except: 
      stock_ats.append("no_data")
    
  mask_store_info_df = pd.DataFrame({"addr":addrs, "code":codes, "latitude":latitudes, "longitude":longitudes, "name":names, "type":types, "created_at":created_ats, "remain_stat":remain_stats, "stock_at":stock_ats}) 
    
  return mask_store_info_df

In [13]:
loc = "관악구"
mask_store_info = getDistrictMaskStoreInfo("서울특별시 "+loc) 
mask_store_info

100%|██████████| 229/229 [00:00<00:00, 197186.54it/s]


,addr,code,latitude,longitude,name,type,created_at,remain_stat,stock_at
0,"서울특별시 관악구 신림로 135 1층 (신림동, 육일빌딩)",11806362,37.471100,126.934334,밝은미래약국,01,2020/07/06 20:35:00,some,2020/07/06 08:52:00
1,서울특별시 관악구 양녕로 38 (봉천동),11809132,37.485329,126.947020,광일약국,01,2020/07/06 20:35:00,some,2020/07/06 07:59:00
2,서울특별시 관악구 양녕로 63 (봉천동),11809191,37.487598,126.946426,문정약국,01,None,None,None
3,서울특별시 관악구 중앙길 5 (봉천동),11809523,37.483630,126.953838,차약국,01,2020/07/06 20:35:00,break,2020/07/06 07:50:00
4,서울특별시 관악구 남부순환로 1471 1층 (신림동),11809710,37.482109,126.914270,허브약국,01,2020/07/06 20:35:00,some,2020/07/06 08:36:00
...,...,...,...,...,...,...,...,...,...
224,서울특별시 관악구 호암로26길 43 1층 (신림동),12870200,37.470464,126.936055,하루약국,01,2020/07/06 20:35:00,some,2020/07/06 18:49:00
225,서울특별시 관악구 관악로 185 1층 (봉천동),12870391,37.481808,126.952672,대동약국,01,2020/07/06 20:35:00,plenty,2020/07/06 10:25:00
226,서울특별시 관악구 남부순환로 1604 1층 (신림동),12870421,37.483946,126.928953,신림4번출구약국,01,2020/07/06 20:35:00,plenty,2020/07/06 09:39:00
227,서울특별시 관악구 관악로 217 동진빌딩 1층 (봉천동),12870439,37.484381,126.954631,웰약국,01,2020/07/06 20:35:00,few,2020/07/06 09:04:00


In [14]:
mask_store_info.dropna(inplace=True)
mask_store_info = mask_store_info[mask_store_info['remain_stat'] != 'no_data']
mask_store = mask_store_info.loc[:, ['code', 'name', 'addr', 'latitude', 'longitude', 'remain_stat', 'stock_at', 'created_at']]
mask_store = mask_store.sort_values(by='remain_stat')
mask_store.reset_index(inplace=True, drop=True)
mask_store

,code,name,addr,latitude,longitude,remain_stat,stock_at,created_at
0,12850080,세영약국,서울특별시 관악구 남부순환로 1479 (신림동),37.482191,126.914955,break,2020/07/06 08:43:00,2020/07/06 20:35:00
1,12835145,수약국,서울특별시 관악구 남부순환로 1912 1층 (봉천동),37.477500,126.961655,break,2020/05/04 07:58:00,2020/05/10 23:55:00
2,12829480,은성온누리약국,서울특별시 관악구 신림로 210 (신림동),37.476140,126.937539,break,2020/07/06 09:23:00,2020/07/06 20:35:00
3,11885548,팜메이트난곡종합약국,서울특별시 관악구 난곡로35길 2 (신림동),37.472969,126.918609,break,2020/07/06 09:11:00,2020/07/06 20:35:00
4,12848387,마더스약국,서울특별시 관악구 난곡로 226 (신림동),37.474524,126.917283,break,2020/07/06 08:58:00,2020/07/06 20:35:00
...,...,...,...,...,...,...,...,...
224,12831964,파운지약국,서울특별시 관악구 참숯1길 4 1층 (봉천동),37.479100,126.942132,None,None,None
225,12836478,태연약국,서울특별시 관악구 난곡로 305 (신림동),37.481245,126.914361,None,None,None
226,12842583,중앙약국,"서울특별시 관악구 난곡로 367 1층 (신림동, 경신빌딩 )",37.486748,126.913357,None,None,None
227,12842869,경희미소원한약국,서울특별시 관악구 난곡로 290 (신림동),37.479878,126.915068,None,None,None


In [19]:
created_at = mask_store['created_at'].mode().to_string()[-19:]
created_at = created_at.replace(':',".").replace("/",".")
data_path = './Data/data_by_district/'
mask_store.to_csv(data_path+loc+" "+created_at+".csv", encoding='utf-8-sig')

## Distribute by district

In [0]:
data_path = './Data/'
date_list = ['snu002-2020-03-14_2020-03-31.csv', 'snu002-2020-04-01_2020-04-15.csv', 'snu002-2020-04-16_2020-04-30.csv', 'snu002-2020-05-01_2020-05-12.csv'] #고려대학교 데이터 hub로부터 제공받은 축적된 데이터
districts = ['강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구','성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구']

raw0 = pd.read_csv(data_path+date_list[0], header=0, engine='python', encoding='utf-8-sig')
raw0.set_index('code', inplace=True, drop=True)
raw1 = pd.read_csv(data_path+date_list[1], header=0, engine='python', encoding='utf-8-sig')
raw1.set_index('code', inplace=True, drop=True)
raw2 = pd.read_csv(data_path+date_list[2], header=0, engine='python', encoding='utf-8-sig')
raw2.set_index('code', inplace=True, drop=True)
raw3 = pd.read_csv(data_path+date_list[3], header=0, engine='python', encoding='utf-8-sig')
raw3.set_index('code', inplace=True, drop=True)

In [0]:
for district in districts:
  df = pd.read_csv(data_path+'data_by_district/'+district+' 2020.05.16 15.00.00.csv', header=0, index_col=[0], engine='python', encoding='utf-8-sig')
  df.set_index('code', inplace=True, drop=True)
  df = df[df['remain_stat'] != 'no_data']
  df.drop(['remain_stat', 'stock_at', 'created_at'], axis=1, inplace=True)

  join0 = df.join(raw0, how='left')
  join1 = df.join(raw1, how='left')
  join2 = df.join(raw2, how='left')
  join3 = df.join(raw3, how='left')

  join_all = pd.concat([join0,join1,join2,join3])
  join_all.reset_index(inplace=True)

  join_all = join_all.groupby('code').apply(lambda x: x.sort_values(['collect_at']))
  join_all.set_index('code', inplace=True, drop=True)
  join_all.to_csv(data_path+'/district_0314-0512/'+district+'_0314-0512.csv', encoding='utf-8-sig')

Visualize mask info

In [1]:
!pip3 install folium

You should consider upgrading via the '/Users/seola/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [16]:
import folium 
from folium.plugins import MarkerCluster, MiniMap

data_for_draw = mask_store.loc[:, ['name', 'latitude', 'longitude', 'remain_stat']]

data_for_draw_except_nan = data_for_draw.dropna() 
data_for_draw_except_nan = data_for_draw_except_nan[data_for_draw_except_nan['remain_stat'] != 'break']

map_mask = folium.Map((37.4710998, 126.9343337), zoom_start = 14)
mc = MarkerCluster()

color_dic = { 'plenty':'green', 'some':'orange', 'few':'red', 'empty':'gray', 'no_data':'purple' }
remain_stat_kor = { 'plenty':'100개 이상', 'some':'30개 이상', "few":'2개이상 30개 미만', "empty":"재고 없음", "no_data":"정보 없음" }

names = list(data_for_draw_except_nan['name'])
latitudes = list(data_for_draw_except_nan['latitude'])
longitudes = list(data_for_draw_except_nan['longitude'])
remain_stats = list(data_for_draw_except_nan['remain_stat'])

for i in tqdm(range(len(names))):
    mc.add_child(folium.Marker(location = [latitudes[i], longitudes[i]], icon=folium.Icon(color=color_dic[remain_stats[i]]), popup=names[i] + ' ' + remain_stat_kor[remain_stats[i]]))
    map_mask.add_child(mc)
    
map_mask.save("./mask_store.html")

100%|██████████| 204/204 [00:00<00:00, 6908.50it/s]


In [20]:
map_mask